---
# Import Libraries

In [70]:
import os
import sys

In [71]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

In [72]:
sys.path.append("../../../deep-learning-dna")
sys.path.append("../")
sys.path.append("../../../deep-learning-dna/common")

In [73]:
import wandb

In [74]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
import math
import string

from Attention import Set_Transformer
from common.models import dnabert
from common import dna
from lmdbm import Lmdb
from common.data import DnaSequenceGenerator, DnaLabelType, DnaSampleGenerator, find_dbs
import wandb

import tf_utils as tfu

In [100]:
strategy = tfu.devices.select_gpu(0, use_dynamic_memory=True)

---
# Load Data

In [101]:
def gen_data(batch_size, length=5):
    x = np.random.randint(1, 100, (batch_size, length))
    y = np.max(x, axis=1)
    return x, y # (batch_size, length)

In [102]:
x, y = gen_data(3, 5)

In [103]:
print(x.shape, y.shape)

(3, 5) (3,)


In [104]:
x

array([[36, 12, 68, 50, 37],
       [43, 29, 98, 57, 65],
       [36, 64, 61, 32, 20]])

In [105]:
y

array([68, 98, 64])

---
# Cache Memory

In [106]:
def Cache_Memory(current_state, previous_state, memory_length):
    
    if memory_length is None or memory_length == 0:
        return None
    else:
        if previous_state is None:
            new_mem = current_state[:, -memory_length:, :]
        else:
            new_mem = tf.concat(
                    [previous_state, current_state], 1)[:, -memory_length:, :]

    return tf.stop_gradient(new_mem)

---
# Attention

In [107]:
class Attention(keras.Model):
    def __init__(self, num_induce, embed_dim, num_heads, use_layernorm, pre_layernorm, use_keras_mha):
        super(Attention, self).__init__()
        
        self.num_induce = num_induce
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.use_layernorm = use_layernorm
        self.pre_layernorm = pre_layernorm
        self.use_keras_mha = use_keras_mha
        
        if self.num_induce == 0:       
            self.attention = (Set_Transformer.SetAttentionBlock(embed_dim=self.embed_dim, num_heads=self.num_heads, use_layernorm=self.use_layernorm,pre_layernorm=self.pre_layernorm,use_keras_mha=self.use_keras_mha))
        else:
            self.attention = Set_Transformer.InducedSetAttentionBlock(embed_dim=self.embed_dim, num_heads=self.num_heads, num_induce=self.num_induce, use_layernorm=self.use_layernorm, pre_layernorm=self.pre_layernorm, use_keras_mha=self.use_keras_mha)
    
    
    def call(self, data, mems):
                
            attention = self.attention([data, mems])
                
            return attention

---
# XL Block

In [108]:
class TransformerXLBlock(tf.keras.layers.Layer):
    def __init__(self,
                 num_induce, 
                 embed_dim,
                 num_heads,
                 use_layernorm,
                 pre_layernorm,
                 use_keras_mha,):

        super(TransformerXLBlock, self).__init__()
        
        self.num_induce = num_induce
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.use_layernorm = use_layernorm
        self.pre_layernorm = pre_layernorm
        self.use_keras_mha = use_keras_mha
        
        self.attention = Attention
        
        self.attention_layer = self.attention(self.num_induce, self.embed_dim, self.num_heads, self.use_layernorm, self.pre_layernorm, self.use_keras_mha)

   
    def call(self,
             content_stream,
             state=None):
        
        attention_output = self.attention_layer(content_stream, state)

        return attention_output

---
# Transformer XL

In [109]:
class TransformerXL(tf.keras.layers.Layer):
    def __init__(self,
                 mem_switched, 
                 num_layers,
                 num_induce,
                 embed_dim,
                 num_heads,
                 dropout_rate,
                 mem_len=None,
                 use_layernorm=True,
                 pre_layernorm=True, 
                 use_keras_mha=True):
        
        super(TransformerXL, self).__init__()

        self.mem_switched = mem_switched
        self.num_layers = num_layers
        self.num_induce = num_induce
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.dropout_rate = dropout_rate
        self.mem_len = mem_len
        self.use_layernorm = use_layernorm
        self.pre_layernorm = pre_layernorm
        self.use_keras_mha = use_keras_mha

        self.transformer_xl_layers = []
        
        for i in range(self.num_layers):
            self.transformer_xl_layers.append(
                    TransformerXLBlock(self.num_induce,
                                        self.embed_dim,
                                        self.num_heads,
                                        self.use_layernorm,
                                        self.pre_layernorm, 
                                        self.use_keras_mha))

        self.output_dropout = tf.keras.layers.Dropout(rate=self.dropout_rate)

    def call(self,
             content_stream,
             state=None):
        
        new_mems = []

        if state is None:
            state = [None] * self.num_layers
            
        for i in range(self.num_layers):
            if self.mem_switched == False:
                new_mems.append(Cache_Memory(content_stream, state[i], self.mem_len))
            
            transformer_xl_layer = self.transformer_xl_layers[i]
            
            transformer_xl_output = transformer_xl_layer(content_stream=content_stream,
                                                        state=state[i])
            
            content_stream = self.output_dropout(transformer_xl_output)
            
            if self.mem_switched == True:
                new_mems.append(Cache_Memory(content_stream, state[i], self.mem_len))

        output_stream = content_stream
        return output_stream, new_mems

---
# Xl Model Class

In [190]:
class XlModel(keras.Model):
    def __init__(self, mem_switched, max_files, encoder, block_size, max_set_len, num_induce, embed_dim, num_layers, num_heads, mem_len, dropout_rate, num_seeds, use_layernorm, pre_layernorm, use_keras_mha):
        super(XlModel, self).__init__()
        
        self.mem_switched = mem_switched
        self.max_files = max_files
        self.encoder = encoder
        self.block_size = block_size
        self.max_set_len = max_set_len
        self.num_induce = num_induce
        self.embed_dim = embed_dim
        self.num_layers = num_layers
        self.num_heads = num_heads
        self.mem_len = mem_len
        self.dropout_rate = dropout_rate
        self.num_seeds = num_seeds
        self.use_layernorm = use_layernorm
        self.pre_layernorm = pre_layernorm
        self.use_keras_mha = use_keras_mha
        
        self.linear_layer = keras.layers.Dense(self.embed_dim)
        
        self.transformer_xl = TransformerXL(self.mem_switched,
                                            self.num_layers,
                                             self.num_induce,
                                             self.embed_dim,
                                             self.num_heads,
                                             self.dropout_rate,
                                             self.mem_len,
                                             self.use_layernorm,
                                             self.pre_layernorm,
                                             self.use_keras_mha)
        

        self.segment_pooling_layer = Set_Transformer.PoolingByMultiHeadAttention(num_seeds=self.num_seeds,embed_dim=self.embed_dim,num_heads=self.num_heads,use_layernorm=self.use_layernorm,pre_layernorm=self.pre_layernorm, use_keras_mha=self.use_keras_mha, is_final_block=True)
       
        self.output_pooling_layer = Set_Transformer.PoolingByMultiHeadAttention(num_seeds=self.num_seeds,embed_dim=self.embed_dim,num_heads=self.num_heads,use_layernorm=self.use_layernorm,pre_layernorm=self.pre_layernorm, use_keras_mha=self.use_keras_mha, is_final_block=True)
    
        self.dropout_layer = keras.layers.Dropout(.5)
    
        self.dense_layer = keras.layers.Dense(100) 
        
        #Classification
        #self.dense_layer = keras.layers.Dense(100) 

    def call(self, x, training=None):    
 
        mems = tf.zeros((self.num_layers, tf.shape(x)[0], self.mem_len, self.embed_dim))
    
        x = tf.expand_dims(x, axis=2)
        
        linear_transform = self.linear_layer(x)

        for i in range(0, self.max_set_len, self.block_size):

            block = linear_transform[:,i:i+self.block_size]
            
            output, mems = self.transformer_xl(content_stream=block, state=mems)
            
            segment_pooling = self.segment_pooling_layer(output)

        output_pooling = self.output_pooling_layer(segment_pooling)
        
        dropout = self.dropout_layer(output_pooling)

        dense = self.dense_layer(dropout)

        #output = tf.reshape(dense, tf.shape(dense)[:2])
        
        #Classification
        output = tf.reshape(dense, tf.shape(dense)[::2])         
        
        return output

---
# Xl Parameters

In [191]:
#Xl Parameters
max_files = 0
encoder = 0
mem_switched = True
num_induce = 0
embed_dim = 64
num_layers = 4
num_heads = 8
mem_len = 0
dropout_rate = 0.01
num_seeds = 1
use_layernorm = True
pre_layernorm = True
use_keras_mha = True

---
# Create Models

In [192]:
block_size = 20
length = 20

In [193]:
x, y = gen_data(batch_size=10, length=length)
vx, vy = gen_data(batch_size=10, length=length)

In [194]:
max_set_len = length
set_len = length

In [195]:
model = XlModel(mem_switched, max_files, encoder, block_size, max_set_len, num_induce, embed_dim, num_layers, num_heads, mem_len, dropout_rate, num_seeds, use_layernorm, pre_layernorm, use_keras_mha)
model.compile(optimizer=keras.optimizers.Adam(1e-3),loss=keras.losses.MeanAbsoluteError())

In [196]:
y[:5]

array([97, 93, 93, 91, 99])

In [197]:
model.predict(x[:5])

<tf.Tensor: shape=(5, 100), dtype=int64, numpy=
array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [198]:
epochs = 500

In [199]:
history = model.fit(x, y, validation_data=(vx, vy), epochs=epochs, verbose=1)

Epoch 1/500
1/1 [==============================] - 4s 4s/step - loss: 96.6529 - val_loss: 95.5401
Epoch 2/500
1/1 [==============================] - 0s 64ms/step - loss: 96.4316 - val_loss: 95.4352
Epoch 3/500
1/1 [==============================] - 0s 58ms/step - loss: 96.2543 - val_loss: 95.3571
Epoch 4/500
1/1 [==============================] - 0s 71ms/step - loss: 96.1945 - val_loss: 95.2737
Epoch 5/500
1/1 [==============================] - 0s 58ms/step - loss: 96.1412 - val_loss: 95.1963
Epoch 6/500
1/1 [==============================] - 0s 58ms/step - loss: 96.1163 - val_loss: 95.1385
Epoch 7/500
1/1 [==============================] - 0s 69ms/step - loss: 95.9952 - val_loss: 95.0952
Epoch 8/500
1/1 [==============================] - 0s 73ms/step - loss: 96.0557 - val_loss: 95.0451
Epoch 9/500
1/1 [==============================] - 0s 70ms/step - loss: 95.8186 - val_loss: 94.9887
Epoch 10/500
1/1 [==============================] - 0s 73ms/step - loss: 95.9525 - val_loss: 94.9337
E

In [174]:
#Training

In [200]:
x[:5]

array([[47,  4, 46, 35,  6, 66, 25, 20, 20, 59, 17, 61,  3, 81, 92, 43,
        98, 84, 16, 49],
       [59,  6, 56, 95, 81, 41, 90, 98, 99, 28, 74, 27, 40, 44, 88, 82,
        34, 35, 52, 97],
       [29, 78, 51, 59, 23, 50, 42, 72, 96, 62, 44, 90, 87, 61, 85, 13,
        23, 44, 87, 36],
       [94, 56, 42, 58, 13, 69, 52, 21, 20, 84, 33, 66, 88, 91, 75, 11,
        76, 36, 87, 94],
       [10, 74, 39, 99, 41, 93, 37, 95,  9, 77, 72, 63, 84, 31, 99, 63,
        94, 37,  3, 28]])

In [201]:
y[:5]

array([98, 99, 96, 94, 99])

In [206]:
model.predict(x[:5])

array([[32.140488, 31.596304, 30.737673, 32.544765, 30.967756, 32.502914,
        30.81377 , 31.895735, 33.349293, 31.751057, 30.733702, 32.056904,
        31.574518, 30.71697 , 32.0162  , 29.719246, 31.637049, 31.282038,
        32.618504, 31.25456 , 30.50701 , 31.72131 , 30.528643, 30.808674,
        31.128302, 32.584743, 30.95209 , 31.915285, 32.5556  , 32.076477,
        30.682821, 32.31818 , 31.379875, 30.952822, 31.354685, 31.11815 ,
        30.438461, 31.566011, 31.844831, 31.607655, 31.682974, 30.332716,
        30.720293, 32.59922 , 31.113966, 31.043636, 29.973253, 32.110764,
        31.531376, 30.71237 , 32.79897 , 30.894453, 30.57506 , 31.708084,
        31.506144, 32.604286, 31.394396, 30.877941, 32.274048, 31.604258,
        30.891043, 30.995413, 31.82604 , 32.312668, 32.039818, 31.06433 ,
        32.422234, 30.516771, 32.598545, 31.704067, 32.62653 , 30.887737,
        32.334496, 31.352152, 31.104177, 31.910337, 31.649847, 31.544945,
        31.999865, 30.358385, 31.49591

In [177]:
#tf.argmax(model.predict(x[:5]), axis = 1)

<tf.Tensor: shape=(5,), dtype=int64, numpy=array([0, 0, 0, 0, 0])>

In [178]:
#Validation

In [179]:
x, y = gen_data(batch_size=30, length=10)

In [ ]:
y[:5]

array([90, 92, 76, 95, 81])

In [181]:
tf.argmax(model.predict(x[:5]), axis = 1)

<tf.Tensor: shape=(5,), dtype=int64, numpy=array([0, 0, 0, 0, 0])>